In [1]:
import polars as pl
import pandas as pd
import plotly.express as px 
import requests
from datetime import datetime
from pathlib import Path
from typing import List, Dict
import os

In [2]:
# constantes
MUSEO_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv"
CINES_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/f7a8edb8-9208-41b0-8f19-d72811dcea97/download/salas_cine.csv"
ESPACIOS_URL="https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/11_bibliotecapopular-datos-abiertos.csv"

list_dicts_url=[
 {
 "name":"museo", "url":MUSEO_URL    
 },
 {
 "name":"cines", "url":CINES_URL    
 },
 {
 "name":"espacios", "url":ESPACIOS_URL    
 }
]

# Path
BASE_FILE_DIR = Path("C:/Users/joan1/OneDrive/Documentos/Programacion/Python/Analisis de datos publicos ARG/tmp").resolve()

# tricks for iterate with a str path (Yuo can define in places holders)
file_path_crib = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.csv"

norm_cols= [
    "cod_localidad",
    "id_provincia",
    "id_departamento",
    "categoría",
    "provincia",
    "localidad",
    "nombre",
    "domicilio",
    "código postal",
    "número de teléfono",
    "mail",
    "web"
]

### Funtions

In [3]:
def get_response_and_create_csv(data_name:dict) -> pl.DataFrame:
    """
        Get the response from the url and create a csv file with the data
    """
    r=requests.get(
        url=data_name["url"]
    )
    r.encoding= "utf-8"

    now = datetime.now()
    category= data_name["name"]

    # create files and fill the places holders
    file_path = BASE_FILE_DIR.joinpath(file_path_crib.format(category=category, 
                                                             year=now.year, 
                                                             month=now.month, 
                                                             day=now.day))
    # create the file
    file_path.parent.mkdir(parents=True, exist_ok=True)

    with open(file=file_path, mode="w") as f:
        f.write(r.text)

    return pl.read_csv(file_path, encoding="latin-1")

def columns_rename(df:pl.DataFrame, old_colums, new_columns):
    """
        Exchange the old columns names for the new columns names
    """
    list_changes = [{old:new} for old, new in zip(old_colums, new_columns)]

    for d in list_changes:
        try:
            df = df.rename(
                d
            )    
        except Exception:
            continue
    return df


### Peticion get para obtener datos

In [4]:
# r=requests.get("https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_4207def0-2ff7-41d5-9095-d42ae8207a5d")
r=requests.get("https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv")
r.encoding= "utf-8"

In [5]:
with open("open","w") as f_out: 
    f_out.write(r.text)

In [6]:
# pd.read_csv("open", delimiter=";", decimal=",",encoding="utf-8")
#pd.read_csv("open",encoding="utf-8-sig")
pd.read_csv("open",encoding="latin-1")

,Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,...,Mail,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
0,6588100,6,6588,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Libertad 1191,...,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,-35.441762,-60.887598,Localización precisa,NaN,DNPyM,Municipal,1920.0,2017
1,6077010,6,6077,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,Gerardo Risso y España,...,museoarrecifes@hotmail.com,NaN,-34.066970,-60.102555,Localización precisa,NaN,DNPyM,Municipal,1972.0,2017
2,6035010,6,6035,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Avellaneda,Museo Histórico-Sacro Hno. Rogelio Scortegagna,Calle 11 (Av. San Martín) Nº 830,...,NaN,NaN,-29.118024,-59.654543,Localización precisa,NaN,DNPyM,Municipal,NaN,2017
3,6035010,6,6035,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Avellaneda,Museo Municipal De Artes Plásticas De Avellaneda,"Sarmiento 101, 1 º Piso",...,NaN,NaN,-34.662110,-58.363270,Localización precisa,NaN,DNPyM,Municipal,1956.0,2017
4,6049020,6,6049,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Azul,Museo Etnográfico Y Archivo Histórico Enrique ...,Bartolomé J. Ronco 654,...,museoazul@yahoo.com,NaN,-36.781824,-59.858636,Localización precisa,NaN,DNPyM,Privada,1949.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,90098040,90,90098,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí del Valle,Museo Histórico Y Arqueológico Jesuítico La Banda,La Banda S/N,...,NaN,NaN,-26.852880,-65.712622,Localización precisa,NaN,DNPyM,NaN,NaN,2017
1178,90105080,90,90105,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Mac-Museo Arqueológico El Cadillal,Centro Principal el Cadillal,...,NaN,NaN,-26.683331,-65.266670,Localización precisa,NaN,DNPyM,Provincial,NaN,2017
1179,90105080,90,90105,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Tafí Viejo,Museo Ferroviario De Los Talleres De Tafí Viejo,Avda. Pte. Roque Sáenz Peña 234,...,NaN,NaN,-26.740733,-65.263231,Localización precisa,NaN,DNPyM,Privada,1909.0,2017
1180,90105070,90,90105,NaN,Espacios de Exhibición Patrimonial,Museos,Tucumán,Villa El Cadillal,Museo Arqueológico Dr. Ernesto Padilla,Ernesto Padilla S/N,...,NaN,NaN,-21.937642,-66.052043,Localización precisa,NaN,DNPyM,Provincial,NaN,2017


In [7]:
pl.read_csv("open", encoding= "latin-1")

Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,piso,CP,cod_area,telefono,Mail,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
i64,i64,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,f64,f64,str,str,str,str,i64,i64
6588100,6,6588,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""9 de Julio""","""Archivo Y Muse…","""Libertad 1191""",null,"""B6500EVL""",2317,"""425 279""","""archivoymuseo@…","""www.portaldel9…",-35.441762,-60.887598,"""Localización p…",null,"""DNPyM""","""Municipal""",1920,2017
6077010,6,6077,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Arrecifes""","""Museo Y Archiv…","""Gerardo Risso …",null,"""B2740FMJ""",2478,"""452 931""","""museoarrecifes…",null,-34.06697,-60.102555,"""Localización p…",null,"""DNPyM""","""Municipal""",1972,2017
6035010,6,6035,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Avellaneda""","""Museo Históric…","""Calle 11 (Av. …",null,"""S3561AKT""",3482,"""481 200""",null,null,-29.118024,-59.654543,"""Localización p…",null,"""DNPyM""","""Municipal""",null,2017
6035010,6,6035,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Avellaneda""","""Museo Municipa…","""Sarmiento 101,…",null,"""B1870CBC""",11,"""4205 9567""",null,null,-34.66211,-58.36327,"""Localización p…",null,"""DNPyM""","""Municipal""",1956,2017
6049020,6,6049,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Azul""","""Museo Etnográf…","""Bartolomé J. R…",null,"""B7300XAA""",2281,"""434 811""","""museoazul@yaho…",null,-36.781824,-59.858636,"""Localización p…",null,"""DNPyM""","""Privada""",1949,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Bella…","""Sarmiento 450""",null,"""B8000HQJ""",291,"""459 4006""","""mac@bblanca.co…",null,-38.7131,-62.2622,"""Localización p…",null,"""DNPyM""","""Municipal""",1935,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Cienc…","""Castelli 3702""",null,"""B8003APV""",291,"""488 3266""",null,null,-38.6928,-62.3087,"""Localización p…",null,"""DNPyM""","""Municipal""",1996,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo Y Parque…","""Italia 19""",null,"""B8000DKA""",291,"""451 0675""",null,null,-38.724527,-62.273735,"""Localización p…",null,"""DNPyM""","""Privada""",2006,2017
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Arte …","""Sarmiento 450""",null,"""B8000HQJ""",291,"""459 4006""","""mac@bblanca.co…",null,-38.7131,-62.2622,"""Localización p…",null,"""DNPyM""","""Municipal""",2008,2017


### museo

In [8]:
df_museo = get_response_and_create_csv(list_dicts_url[0])

In [9]:
df_museo.describe()

describe,Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,piso,CP,cod_area,telefono,Mail,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
str,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,f64,str,str,str,str,f64,str
"""count""",1182.0,1182.0,1182.0,"""1182""","""1182""","""1182""","""1182""","""1182""","""1182""","""1182""","""1182""","""1182""",1182.0,"""1182""","""1182""","""1182""",1182.0,1182.0,"""1182""","""1182""","""1182""","""1182""",1182.0,"""1182"""
"""null_count""",0.0,0.0,0.0,"""1182""","""0""","""133""","""0""","""0""","""0""","""18""","""1182""","""66""",193.0,"""194""","""918""","""1107""",0.0,0.0,"""0""","""1172""","""0""","""97""",539.0,"""0"""
"""mean""",3.8823e7,38.717428,38822.78511,null,null,null,null,null,null,null,null,null,1623.187058,null,null,null,-33.140116,-62.718226,null,null,null,null,1758.615863,null
"""std""",3.0024e7,30.073683,30024.414238,null,null,null,null,null,null,null,null,null,1542.080323,null,null,null,5.437096,4.003067,null,null,null,null,620.695949,null
"""min""",2.00001e6,2.0,2000.0,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""25 de Mayo""","""Academia Argen…","""1 de Mayo 2152…",null,"""1104""",11.0,"""15 4047 509""","""Amupri2@yahoo.…","""/aafsantafe.co…",-54.871872,-72.334078,"""Centroide por …","""Casas de la Hi…","""CNMLH - Enlace…","""Comunal""",0.0,"""2017"""
"""25%""",1.003515e7,10.0,10035.0,null,null,null,null,null,null,null,null,null,264.0,null,null,null,-34.6565,-65.416138,null,null,null,null,1946.0,null
"""50%""",3.401402e7,34.0,34014.0,null,null,null,null,null,null,null,null,null,387.0,null,null,null,-32.857792,-62.273735,null,null,null,null,1981.0,null
"""75%""",6.611204e7,66.0,66112.0,null,null,null,null,null,null,null,null,null,3404.0,null,null,null,-30.914302,-58.710025,null,null,null,null,1998.0,null
"""max""",9.401402e7,94.0,94014.0,null,"""Espacios de Ex…","""Museos""","""Tucumán""","""al Norte de In…","""Yuchan""","""Árabe Siria y …",null,"""Z9405AHD""",8522.0,"""981 518""","""zoologico@mend…","""www.wagnermuse…",-21.937642,-54.114682,"""Localización p…","""Templo Religio…","""Área Gestión C…","""s/d""",2017.0,"""2017 """


In [10]:
df_museo.columns

['Cod_Loc',
 'IdProvincia',
 'IdDepartamento',
 'Observaciones',
 'categoria',
 'subcategoria',
 'provincia',
 'localidad',
 'nombre',
 'direccion',
 'piso',
 'CP',
 'cod_area',
 'telefono',
 'Mail',
 'Web',
 'Latitud',
 'Longitud',
 'TipoLatitudLongitud',
 'Info_adicional',
 'fuente',
 'jurisdiccion',
 'año_inauguracion',
 'actualizacion\r']

In [11]:
need_columns = [
'Cod_Loc',
 'IdProvincia',
 'IdDepartamento',
#  'Observaciones',
 'categoria',
#  'subcategoria',
 'provincia',
 'localidad',
 'nombre',
 'direccion',
#  'piso',
 'CP',
# 'cod_area',
 'telefono',
 'Mail',
 'Web',
#  'Latitud',
#  'Longitud',
#  'TipoLatitudLongitud',
#  'Info_adicional',
#  'fuente',
#  'jurisdiccion',
#  'año_inauguracion',
#  'actualizacion\r'
]

#### normalizar las columnas

In [12]:
list_changes = [{old:new} for old, new in zip(need_columns, norm_cols)]
list_changes

[{'Cod_Loc': 'cod_localidad'},
 {'IdProvincia': 'id_provincia'},
 {'IdDepartamento': 'id_departamento'},
 {'categoria': 'categoría'},
 {'provincia': 'provincia'},
 {'localidad': 'localidad'},
 {'nombre': 'nombre'},
 {'direccion': 'domicilio'},
 {'CP': 'código postal'},
 {'telefono': 'número de teléfono'},
 {'Mail': 'mail'},
 {'Web': 'web'}]

In [13]:
for d in list_changes:
   df_museo = df_museo.rename(
        d
    )

In [14]:
df_museo

cod_localidad,id_provincia,id_departamento,Observaciones,categoría,subcategoria,provincia,localidad,nombre,domicilio,piso,código postal,cod_area,número de teléfono,mail,web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
i64,i64,i64,str,str,str,str,str,str,str,str,str,i64,str,str,str,f64,f64,str,str,str,str,i64,str
6588100,6,6588,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""9 de Julio""","""Archivo Y Muse…","""Libertad 1191""",null,"""B6500EVL""",2317,"""425 279""","""archivoymuseo@…","""www.portaldel9…",-35.441762,-60.887598,"""Localización p…",null,"""DNPyM""","""Municipal""",1920,"""2017 """
6077010,6,6077,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Arrecifes""","""Museo Y Archiv…","""Gerardo Risso …",null,"""B2740FMJ""",2478,"""452 931""","""museoarrecifes…",null,-34.06697,-60.102555,"""Localización p…",null,"""DNPyM""","""Municipal""",1972,"""2017 """
6035010,6,6035,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Avellaneda""","""Museo Históric…","""Calle 11 (Av. …",null,"""S3561AKT""",3482,"""481 200""",null,null,-29.118024,-59.654543,"""Localización p…",null,"""DNPyM""","""Municipal""",null,"""2017 """
6035010,6,6035,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Avellaneda""","""Museo Municipa…","""Sarmiento 101,…",null,"""B1870CBC""",11,"""4205 9567""",null,null,-34.66211,-58.36327,"""Localización p…",null,"""DNPyM""","""Municipal""",1956,"""2017 """
6049020,6,6049,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Azul""","""Museo Etnográf…","""Bartolomé J. R…",null,"""B7300XAA""",2281,"""434 811""","""museoazul@yaho…",null,-36.781824,-59.858636,"""Localización p…",null,"""DNPyM""","""Privada""",1949,"""2017 """
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Bella…","""Sarmiento 450""",null,"""B8000HQJ""",291,"""459 4006""","""mac@bblanca.co…",null,-38.7131,-62.2622,"""Localización p…",null,"""DNPyM""","""Municipal""",1935,"""2017 """
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Cienc…","""Castelli 3702""",null,"""B8003APV""",291,"""488 3266""",null,null,-38.6928,-62.3087,"""Localización p…",null,"""DNPyM""","""Municipal""",1996,"""2017 """
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo Y Parque…","""Italia 19""",null,"""B8000DKA""",291,"""451 0675""",null,null,-38.724527,-62.273735,"""Localización p…",null,"""DNPyM""","""Privada""",2006,"""2017 """
6056010,6,6056,null,"""Espacios de Ex…","""Museos""","""Buenos Aires""","""Bahía Blanca""","""Museo De Arte …","""Sarmiento 450""",null,"""B8000HQJ""",291,"""459 4006""","""mac@bblanca.co…",null,-38.7131,-62.2622,"""Localización p…",null,"""DNPyM""","""Municipal""",2008,"""2017 """


### cines

In [15]:
df_cines = get_response_and_create_csv(list_dicts_url[1])

In [16]:
df_cines.sample(5)

cod_localidad,id_provincia,id_departamento,categoria,provincia,departamento,localidad,nombre,direccion,piso,cp,web,latitud,longitud,tipo_latitud_longitud,fuente,sector,pantallas,butacas,tipo_de_gestion,espacio_incaa,año_actualizacion
i64,i64,i64,str,str,str,str,str,str,str,i64,str,f64,f64,str,str,str,i64,i64,str,str,str
6427010,6,6427,"""Salas de cine""","""Buenos Aires""","""La Matanza""","""Gonzalez Catán…","""CPM Catan""","""Ruta 3 Km 29 B…",null,1759,"""https://cpmcin…",-34.761032,-58.615582,"""Precisa""","""INCAA / SInCA""","""Cine""",6,1114,"""Privada""","""No""","""2022 """
6770050,6,6770,"""Salas de cine""","""Buenos Aires""","""San Pedro""","""San Pedro""","""La Palma""","""Belgrano 961""",null,2930,"""https://www.fa…",-33.678826,-59.668011,"""Precisa""","""INCAA / SInCA""","""Cine""",2,471,"""Privada""","""No""","""2022 """
78021040,78,78021,"""Salas de cine""","""Santa Cruz""","""Guer Aike""","""Río Gallegos""","""Cine Rio Galle…","""Av. Nestor Kir…",null,9400,"""http://www.cin…",-51.619917,-69.218735,"""Precisa""","""INCAA / SInCA""","""Cine""",3,290,"""Privada""","""No""","""2022 """
6441030,6,6441,"""Salas de cine""","""Buenos Aires""","""La Plata""","""La Plata""","""Cine Municipal…","""19 Y 51""","""Piso 2""",1900,"""http://www.cul…",-34.926546,-57.961577,"""Precisa""","""INCAA / SInCA""","""Cine""",1,60,"""Pública""","""Si""","""2022 """
6441030,6,6441,"""Salas de cine""","""Buenos Aires""","""La Plata""","""La Plata""","""Cine Municipal…","""Calle 50 entre…",null,1900,"""http://www.cul…",-34.91424,-57.949226,"""Precisa""","""INCAA / SInCA""","""Cine""",1,174,"""Pública""","""Si""","""2022 """


In [17]:
df_cines.columns

['cod_localidad',
 'id_provincia',
 'id_departamento',
 'categoria',
 'provincia',
 'departamento',
 'localidad',
 'nombre',
 'direccion',
 'piso',
 'cp',
 'web',
 'latitud',
 'longitud',
 'tipo_latitud_longitud',
 'fuente',
 'sector',
 'pantallas',
 'butacas',
 'tipo_de_gestion',
 'espacio_incaa',
 'año_actualizacion\r']

In [18]:
new_columns = [i for i in norm_cols if i not in df_cines.columns]
new_columns.remove("número de teléfono")
new_columns.remove("mail")
new_columns

['categoría', 'domicilio', 'código postal']

In [19]:

old_columns = ['categoria',
 'direccion',
 'cp',
 'web',]

In [20]:
df_cines=columns_rename(df_cines, old_columns, new_columns)
df_cines = df_cines.with_columns(df_cines["código postal"].cast(str))
df_cines

cod_localidad,id_provincia,id_departamento,categoría,provincia,departamento,localidad,nombre,domicilio,piso,código postal,web,latitud,longitud,tipo_latitud_longitud,fuente,sector,pantallas,butacas,tipo_de_gestion,espacio_incaa,año_actualizacion
i64,i64,i64,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,i64,i64,str,str,str
6854100,6,6854,"""Salas de cine""","""Buenos Aires""","""25 de mayo""","""25 de mayo""","""Cine Teatro Es…","""Calle 9 entre …",null,"""6660""","""https://www.in…",-35.430488,-60.170686,"""Precisa""","""INCAA / SInCA""","""Cine""",1,500,"""Pública""","""No""","""2022 """
6826040,6,6826,"""Salas de cine""","""Buenos Aires""","""Trenque Lauque…","""30 de agosto""","""Cine Santa Ros…","""Santa Rosa De …",null,"""6405""","""https://www.tr…",-36.278508,-62.544355,"""Precisa""","""INCAA / SInCA""","""Cine""",1,410,"""Pública""","""No""","""2022 """
6588100,6,6588,"""Salas de cine""","""Buenos Aires""","""9 de julio""","""9 de julio""","""Tu Cine 9 de J…","""Robbio 840""",null,"""6500""","""http://9dejuli…",-35.446265,-60.885789,"""Precisa""","""INCAA / SInCA""","""Cine""",1,165,"""Privada""","""No""","""2022 """
6028010,6,6028,"""Salas de cine""","""Buenos Aires""","""Almirante Brow…","""Adrogué""","""Cinema Adrogue…","""Hipolito Yrigo…",null,"""1846""","""https://www.ci…",-34.80811,-58.4024,"""Precisa""","""INCAA / SInCA""","""Cine""",10,2505,"""Privada""","""No""","""2022 """
6028010,6,6028,"""Salas de cine""","""Buenos Aires""","""Almirante Brow…","""Adrogué""","""CPM Paseo Adro…","""Segui 699""",null,"""1846""","""https://cpmcin…",-34.800097,-58.396753,"""Precisa""","""INCAA / SInCA""","""Cine""",4,710,"""Privada""","""No""","""2022 """
6035010,6,6035,"""Salas de cine""","""Buenos Aires""","""Avellaneda""","""Avellaneda""","""Cinemark Alto …","""Guemes 897""",null,"""1870""","""https://www.ci…",-34.677993,-58.36723,"""Precisa""","""INCAA / SInCA""","""Cine""",6,1275,"""Privada""","""No""","""2022 """
6035010,6,6035,"""Salas de cine""","""Buenos Aires""","""Avellaneda""","""Avellaneda""","""Cinepolis Avel…","""Autopista Bs. …",null,"""1870""","""https://www.ci…",-34.680206,-58.332777,"""Precisa""","""INCAA / SInCA""","""Cine""",16,3485,"""Privada""","""No""","""2022 """
6049020,6,6049,"""Salas de cine""","""Buenos Aires""","""Azul""","""Azul""","""Flix Cinema""","""Bolivar 416""",null,"""7300""","""http://azul.fl…",-36.77836,-59.864783,"""Precisa""","""INCAA / SInCA""","""Cine""",1,160,"""Privada""","""No""","""2022 """
6056010,6,6056,"""Salas de cine""","""Buenos Aires""","""Bahía Blanca""","""Bahía Blanca""","""Cine Vision""","""Belgrano 153""",null,"""8000""","""https://www.ci…",-38.718149,-62.262122,"""Precisa""","""INCAA / SInCA""","""Cine""",3,907,"""Privada""","""No""","""2022 """


### Espacios

In [21]:
df_espacios = get_response_and_create_csv(list_dicts_url[2])

In [22]:
new_columns = [i for i in norm_cols if i not in df_espacios.columns]
new_columns

['categoría', 'código postal', 'número de teléfono']

In [23]:
df_espacios.columns

['cod_localidad',
 'id_provincia',
 'id_departamento',
 'observacion',
 'categoria',
 'subcategoria',
 'provincia',
 'departamento',
 'localidad',
 'nombre',
 'domicilio',
 'piso',
 'cp',
 'cod_tel',
 'telefono',
 'mail',
 'web',
 'informacion_adicional',
 'latitud',
 'longitud',
 'tipo_latitud_longitud',
 'fuente',
 'fecha_fundacion',
 'nro_conabip',
 'anio_actualizacion\r']

In [24]:
old_columns = ['categoria',
 'cp',
 'telefono',]
df_espacios=columns_rename(df_espacios, old_columns, new_columns)

In [25]:
df_espacios

cod_localidad,id_provincia,id_departamento,observacion,categoría,subcategoria,provincia,departamento,localidad,nombre,domicilio,piso,código postal,cod_tel,número de teléfono,mail,web,informacion_adicional,latitud,longitud,tipo_latitud_longitud,fuente,fecha_fundacion,nro_conabip,anio_actualizacion
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,f64,f64,str,str,str,i64,str
6854100,6,6854,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""25 De Mayo""","""25 de Mayo""","""Comité Cultura…","""9 y 27 801""",null,"""6660""",2345,462234,null,null,null,-35.432917,-60.171417,"""Exacta""","""CONABIP""","""1922-10-02""",938,"""2023 """
6588100,6,6588,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""9 De Julio""","""9 de Julio""","""Bib.Pop. Anast…","""Mitre 342""",null,"""6500""",null,null,null,null,null,-35.433333,-60.866667,"""Exacta""","""CONABIP""","""1980-01-01""",200,"""2023 """
6588100,6,6588,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""9 De Julio""","""9 de Julio""","""Bib.Pop. José …","""Mendoza 991""",null,"""6500""",2317,426779,null,null,null,-35.445957,-60.883377,"""Exacta""","""CONABIP""","""1933-10-12""",1850,"""2023 """
6007010,6,6007,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Adolfo Alsina""","""Carhue""","""Bib.Pop. Adolf…","""San Martín 110…",null,"""B6430CSL""",2936,430724,null,null,null,-37.180933,-62.76005,"""Exacta""","""CONABIP""","""1912-08-23""",88,"""2023 """
6007080,6,6007,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Adolfo Alsina""","""Rivera""","""Bib.Pop. José …","""Los Colonizado…",null,"""6441""",2935,432073,"""ccibiblio@hotm…",null,null,-37.1626,-63.24705,"""Exacta""","""CONABIP""","""1925-01-01""",1278,"""2023 """
6014010,6,6014,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Adolfo Gonzale…","""Adolfo Gonzale…","""Bib.Pub. Berna…","""Sarmiento 175""",null,"""7513""",2983,484115,"""bipobrchaves@y…",null,null,-38.03465,-60.099433,"""Exacta""","""CONABIP""","""1925-05-10""",1062,"""2023 """
6014020,6,6014,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Adolfo Gonzale…","""De la Garma""","""Bib.Pop. Dr. V…","""Castro Barros …",null,"""B7515AGH""",2983,494240,"""barrerabp@hotm…",null,null,-37.962467,-60.414517,"""Exacta""","""CONABIP""","""1942-11-30""",2170,"""2023 """
6021010,6,6021,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Alberti""","""Alberti""","""Bib.Pop. André…","""Arias 154""",null,"""6634""",2346,471339,"""eleana_1983@ho…",null,null,-35.03105,-60.278333,"""Exacta""","""CONABIP""","""1925-01-01""",1554,"""2023 """
6028010,6,6028,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Almte Brown""","""Adrogué""","""Amigos de la B…","""La Rosa 974 0""",null,"""B1846CQN""",11,42141245,null,null,null,-34.797233,-58.38155,"""Exacta""","""CONABIP""","""1918-08-01""",90,"""2023 """


### Registros totales por categoria

In [26]:
df_espacios.columns

['cod_localidad',
 'id_provincia',
 'id_departamento',
 'observacion',
 'categoría',
 'subcategoria',
 'provincia',
 'departamento',
 'localidad',
 'nombre',
 'domicilio',
 'piso',
 'código postal',
 'cod_tel',
 'número de teléfono',
 'mail',
 'web',
 'informacion_adicional',
 'latitud',
 'longitud',
 'tipo_latitud_longitud',
 'fuente',
 'fecha_fundacion',
 'nro_conabip',
 'anio_actualizacion\r']

In [27]:
list_df: List[pl.DataFrame]=[df_museo,df_cines,df_espacios]

list_colums = ["código postal", 'número de teléfono', "mail", "web"]

#normalize columns type
for df in list_df:
    for col in list_colums:
        if col in df.columns:
            try:
                df = df.with_columns(pl.col(col).cast(str))
            except Exception as e:
                # polars errors cant be catched with try except
                print("error")


In [30]:
df_espacios = df_espacios.with_columns(pl.col(list_colums[1]).cast(str))

In [33]:
df_espacios.sample(5)

cod_localidad,id_provincia,id_departamento,observacion,categoría,subcategoria,provincia,departamento,localidad,nombre,domicilio,piso,código postal,cod_tel,número de teléfono,mail,web,informacion_adicional,latitud,longitud,tipo_latitud_longitud,fuente,fecha_fundacion,nro_conabip,anio_actualizacion
i64,i64,i64,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,f64,f64,str,str,str,i64,str
6833060,6,6833,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Tres Arroyos""","""Orense""","""Bib.Pop.Pub. A…","""J.P.de Riglos …",null,"""B7503AZA""",2982,"""420101""","""catriel_h@hotm…",null,null,-38.687417,-59.782333,"""Exacta""","""CONABIP""","""1927-08-27""",1299,"""2023 """
26105010,26,26105,null,"""Biblioteca Pop…",null,"""Chubut""","""Telsen""","""Gan-Gan""","""Bib.Pop. Narlu…","""Av.Omar A.Mont…",null,"""9121""",0,"""0""",null,null,null,-42.521117,-68.2851,"""Exacta""","""CONABIP""","""1980-01-01""",3584,"""2023 """
82091220,82,82091,null,"""Biblioteca Pop…",null,"""Santa Fe""","""San Cristobal""","""San Cristóbal""","""Bib.Pop. San M…","""Alvear 646""",null,"""3070""",3408,"""423835""","""bibliotecapop_…",null,null,-30.31085,-61.23295,"""Exacta""","""CONABIP""","""1910-12-03""",460,"""2023 """
6875060,6,6875,null,"""Biblioteca Pop…",null,"""Buenos Aires""","""Villariño""","""Médanos""","""Bib.Pop. Barto…","""Bartolomé Mitr…",null,"""8132""",2927,"""432003""","""bbcamitre@yaho…",null,null,-38.82858,-62.694325,"""Exacta""","""CONABIP""","""1939-04-24""",2040,"""2023 """
14182210,14,14182,null,"""Biblioteca Pop…",null,"""Córdoba""","""Union""","""Pascanas""","""Bib.Pop. Pasca…","""Rivadavia 258""",null,"""2679""",353,"""4898075""","""atleticopascan…",null,null,-33.1247,-63.03925,"""Exacta""","""CONABIP""","""1931-01-01""",1837,"""2023 """


In [28]:
[i for i in df_museo.columns if i in df_espacios.columns]

['cod_localidad',
 'id_provincia',
 'id_departamento',
 'categoría',
 'subcategoria',
 'provincia',
 'localidad',
 'nombre',
 'domicilio',
 'piso',
 'código postal',
 'número de teléfono',
 'mail',
 'web',
 'fuente']

In [32]:

# concatenate dfs
pl.concat(list_df, how="diagonal")

ShapeError: unable to vstack, dtypes for column "número de teléfono" don't match: `str` and `i64`

In [ ]:
df_registers_totales_by_category

categoría,registros_totales
str,i64
"""museo""",1182
"""cines""",316
"""espacios""",1902


### registros totales por provincia